In [3]:
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input, Dense, Flatten, Embedding

Using TensorFlow backend.


In [4]:
# define documents
docs = ['Well done!',
        'Good work',
        'Great effort',
        'nice work',
        'Excellent!',
        'Weak',
        'Poor effort!',
        'not good',
        'poor work',
        'Could have done better.']
# define class labels
labels = np.array([1,1,1,1,1,0,0,0,0,0])

In [5]:
# this can be used either
tokenizer = Tokenizer()
tokenizer.fit_on_texts(docs)
vocab_size = len(tokenizer.word_index) + 1
encoded_docs = tokenizer.texts_to_sequences(docs)
print(encoded_docs)
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

[[6, 2], [3, 1], [7, 4], [8, 1], [9], [10], [5, 4], [11, 3], [5, 1], [12, 13, 2, 14]]
[[ 6  2  0  0]
 [ 3  1  0  0]
 [ 7  4  0  0]
 [ 8  1  0  0]
 [ 9  0  0  0]
 [10  0  0  0]
 [ 5  4  0  0]
 [11  3  0  0]
 [ 5  1  0  0]
 [12 13  2 14]]


In [6]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('glove.6B.100d.txt', encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.array(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [7]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [8]:
embedding = Embedding(vocab_size, 100, weights=[embedding_matrix], trainable=False)

W0903 16:25:14.274927  1748 deprecation_wrapper.py:119] From C:\Users\10\Anaconda3\envs\gpu_env\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



In [9]:
inputs = Input(shape=(4,), name='inputs')
x = inputs
x = embedding(x)
x = Flatten()(x)
outputs = Dense(1, activation='sigmoid')(x)
model = Model(inputs, outputs, name='simple_embedding')
model.summary()
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])

W0903 16:25:14.299859  1748 deprecation_wrapper.py:119] From C:\Users\10\Anaconda3\envs\gpu_env\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0903 16:25:14.303853  1748 deprecation_wrapper.py:119] From C:\Users\10\Anaconda3\envs\gpu_env\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0903 16:25:14.313822  1748 deprecation_wrapper.py:119] From C:\Users\10\Anaconda3\envs\gpu_env\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0903 16:25:14.313822  1748 deprecation_wrapper.py:119] From C:\Users\10\Anaconda3\envs\gpu_env\lib\site-packages\keras\backend\tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0903 16:25:15.696741  1748

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 4)                 0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 4, 100)            1500      
_________________________________________________________________
flatten_1 (Flatten)          (None, 400)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 401       
Total params: 1,901
Trainable params: 401
Non-trainable params: 1,500
_________________________________________________________________


In [10]:
model.fit(padded_docs,
          labels,
          epochs=100)
loss, acc = model.evaluate(padded_docs, labels)
print('Accuracy: ', acc * 100)

Epoch 1/100
10/10 [==============================] - 0s 44ms/step - loss: 0.8835 - acc: 0.3000
Epoch 2/100
10/10 [==============================] - 0s 299us/step - loss: 0.8671 - acc: 0.3000
Epoch 3/100
10/10 [==============================] - 0s 399us/step - loss: 0.8511 - acc: 0.3000
Epoch 4/100
10/10 [==============================] - 0s 299us/step - loss: 0.8354 - acc: 0.3000
Epoch 5/100
10/10 [==============================] - 0s 296us/step - loss: 0.8201 - acc: 0.3000
Epoch 6/100
10/10 [==============================] - 0s 299us/step - loss: 0.8050 - acc: 0.3000
Epoch 7/100
10/10 [==============================] - 0s 397us/step - loss: 0.7904 - acc: 0.4000
Epoch 8/100
10/10 [==============================] - 0s 398us/step - loss: 0.7760 - acc: 0.4000
Epoch 9/100
10/10 [==============================] - 0s 399us/step - loss: 0.7620 - acc: 0.4000
Epoch 10/100
10/10 [==============================] - 0s 299us/step - loss: 0.7484 - acc: 0.4000
Epoch 11/100
10/10 [====================

Epoch 86/100
10/10 [==============================] - 0s 299us/step - loss: 0.2761 - acc: 1.0000
Epoch 87/100
10/10 [==============================] - 0s 199us/step - loss: 0.2733 - acc: 1.0000
Epoch 88/100
10/10 [==============================] - 0s 299us/step - loss: 0.2705 - acc: 1.0000
Epoch 89/100
10/10 [==============================] - 0s 199us/step - loss: 0.2677 - acc: 1.0000
Epoch 90/100
10/10 [==============================] - 0s 299us/step - loss: 0.2650 - acc: 1.0000
Epoch 91/100
10/10 [==============================] - 0s 199us/step - loss: 0.2623 - acc: 1.0000
Epoch 92/100
10/10 [==============================] - 0s 199us/step - loss: 0.2596 - acc: 1.0000
Epoch 93/100
10/10 [==============================] - 0s 399us/step - loss: 0.2570 - acc: 1.0000
Epoch 94/100
10/10 [==============================] - 0s 200us/step - loss: 0.2545 - acc: 1.0000
Epoch 95/100
10/10 [==============================] - 0s 299us/step - loss: 0.2519 - acc: 1.0000
Epoch 96/100
10/10 [==========

## Let's test the model

In [11]:
sent = 'Well work!'
encoded_docs = tokenizer.texts_to_sequences([sent])
padded_encoded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post' )
padded_encoded_docs

array([[6, 1, 0, 0]])

In [14]:
model.predict(padded_encoded_docs)

array([[0.72876847]], dtype=float32)

*:)*